In [1]:
import torch
from torchvision import datasets, transforms
from WongBasedTraining import WongBasedTrainingCIFAR10
from Architectures import PreActResNet18
from Ensemble import Ensemble
from AdversarialAttacks import attack_pgd
import numpy as np
import os
import csv
from Testing import testEnsemble

In [2]:
train_config = {
    "num_samples_wl": 750000,
    "num_wl": 15,
    "train_eps_wl": 8,
    "training_valrestarts": 1,
    "dataset_name": "cifar10",
    "train_alpha_wl": 10,
    "lr_min_wl": 0,
    "lr_max_wl": 0.2,
    "weight_decay_wl": 5e-4,
    "early_stop_wl": True,
    "delta_init_wl": "random",
    "momentum_wl": 0.9,
    "seed_wl": None,
    "opt_level_wl": "O2",
    "loss_scale_wl": 1.0,
    "delta_init_wl": "random",
    "momentum_wl": 0.9,
    "attack_iters_val_wl": 20,
    "batch_size_wl": 128,
    "restarts_wl": 1,
    "training_method": "wong",
    "adv_train_wl": True,
    "prediction_weights_wl": False,
    "lr_schedule_wl": "cyclic",
    "attack_eps_wl": [0.127],
    "model_base": PreActResNet18,
    "val_attacks": [attack_pgd],
    "dataset": datasets.CIFAR10,
    "weak_learner_type": WongBasedTraining
}
weakLearners = []
weakLearnerWeights = []

In [3]:
# from ipywidgets import IntProgress
from utils import applyDSTrans
train_ds, test_ds = applyDSTrans(train_config)
train_ds.targets = torch.tensor(np.array(train_ds.targets))
test_ds.targets = torch.tensor(np.array(test_ds.targets))
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=128, shuffle=False)
path_head = f"./models/{train_config['training_method']}/{train_config['dataset_name']}/baseline/{train_config['num_samples_wl']}Eps{train_config['train_eps_wl']}/"
if os.path.exists(path_head):
    print("Already exists, exiting")
else:
    os.mkdir(path_head)

(0.4914, 0.4822, 0.4465) (0.2471, 0.2435, 0.2616)
Normalized DS
Files already downloaded and verified
Files already downloaded and verified


In [4]:
for t in range(train_config['num_wl']):
    h_i = train_config['weak_learner_type'](model_base=train_config['model_base'], attack_eps=train_config['attack_eps_wl'])
    h_i.fit(train_loader, test_loader, train_config)
    model_path = f'{path_head}wl_{t}.pth'
    torch.save(h_i.model.state_dict(), model_path)
    weakLearners.append(model_path)
    print("weakLearnerWeights before:", weakLearnerWeights)
    weakLearnerWeights.append(1.)
    print("weakLearnerWeights after:", weakLearnerWeights)

Epoch 	 Seconds 	 LR 	 	 Train Loss 	 Train Acc
train_loader: <torch.utils.data.dataloader.DataLoader object at 0x7fa031f9edf0>
Progress: 12800,  val accuracy: 0.3203
PGD accuracy: [[0.21875]]
Progress: 25600,  val accuracy: 0.2969
PGD accuracy: [[0.21875, 0.1640625]]
Progress: 38400,  val accuracy: 0.4141
PGD accuracy: [[0.21875, 0.1640625, 0.21875]]
0 	 125.9 	 	 0.0000 	 2.0699 	 0.2210
Total train time: %.4f minutes 2.098653837045034
weakLearnerWeights before: []
weakLearnerWeights after: [1.0]
Epoch 	 Seconds 	 LR 	 	 Train Loss 	 Train Acc
train_loader: <torch.utils.data.dataloader.DataLoader object at 0x7fa031f9edf0>
Progress: 12800,  val accuracy: 0.3594
PGD accuracy: [[0.25]]
Progress: 25600,  val accuracy: 0.3203
PGD accuracy: [[0.25, 0.203125]]
Progress: 38400,  val accuracy: 0.4062
PGD accuracy: [[0.25, 0.203125, 0.234375]]
0 	 125.4 	 	 0.0000 	 2.0611 	 0.2266
Total train time: %.4f minutes 2.09003590742747
weakLearnerWeights before: [1.0]
weakLearnerWeights after: [1.0, 

In [5]:
weight_path = f"{path_head}wl_weights.csv"
with open(weight_path, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(weakLearnerWeights)

In [6]:
test_config = {
    "num_samples_wl": 750000,
    "num_samples_train": 200,
    "num_samples_val": 1500,
    "train_eps_wl": 8,
    "num_wl": 15,
    "batch_size": 128,
    "testing_attack_iters": 20,
    "testing_restarts": 10,
    "dataset_name": "cifar10",
    "training_method": "wong",
    "train_batch_size": 128,
    "test_batch_size": 128,
    "attack_eps_ensemble": [0.127],
    "auto_attack": True,
    "model_base": PreActResNet18,
    "dataset": datasets.CIFAR10,
    'weak_learner_type': WongBasedTrainingCIFAR10,
    'path': path_head
}


In [7]:
test_config['results_path'] = f"results/plots/{test_config['training_method']}/{test_config['dataset_name']}/baseline/train_eps_{test_config['train_eps_wl']}"

In [8]:
testEnsemble(test_config)

(0.4914, 0.4822, 0.4465) (0.2471, 0.2435, 0.2616)
Files already downloaded and verified
Files already downloaded and verified
Weak Learner  0 .  Time Elapsed (s):  1
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square
initial accuracy: 42.48%


/opt/conda/lib/python3.8/site-packages/autoattack/autopgd_base.py:506: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:983.)
  ind_to_fool = acc.nonzero().squeeze()


apgd-ce - 1/4 - 53 out of 128 successfully perturbed
apgd-ce - 2/4 - 44 out of 128 successfully perturbed
apgd-ce - 3/4 - 48 out of 128 successfully perturbed
apgd-ce - 4/4 - 13 out of 51 successfully perturbed
robust accuracy after APGD-CE: 27.05% (total time 127.5 s)
apgd-t - 1/3 - 16 out of 128 successfully perturbed
apgd-t - 2/3 - 15 out of 128 successfully perturbed
apgd-t - 3/3 - 4 out of 21 successfully perturbed
robust accuracy after APGD-T: 23.63% (total time 923.5 s)
fab-t - 1/2 - 0 out of 128 successfully perturbed
fab-t - 2/2 - 0 out of 114 successfully perturbed
robust accuracy after FAB-T: 23.63% (total time 1956.9 s)
square - 1/2 - 0 out of 128 successfully perturbed
square - 2/2 - 0 out of 114 successfully perturbed
robust accuracy after SQUARE: 23.63% (total time 4344.2 s)
max Linf perturbation: 0.03100, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 23.63%
Weak Learner  1 .  Time Elapsed (s):  4347
setting parameters for standard version
using standard 

KeyboardInterrupt: 

In [ ]:
print(weakLearnerWeights)